In [1]:
import ray
import json
import pandas as pd
import time

In [24]:
ray.init(num_cpus=4,resources={'CustomResource1': 1, 'CustomResource2': 4},
         include_webui=False, ignore_reinit_error=True)

Calling ray.init() again after it has already been called.


In [7]:
with open('Data.json','r') as f:
    data_all = f.read()
    
data = json.loads(data_all)

In [8]:
#This is the list of Text
X = list(data['Text'].values())
#This is the list of Manual Sentiment
Y = list(data['Sentiment'].values())
#This is the list for Company name
Z = list(data['Company'].values())

In [9]:
Man_Senti = pd.DataFrame({'Text':X, 'Sentiment': Y, 'Company': Z})

In [10]:
Man_Senti.head()

,Company,Sentiment,Text
0,Google,Neutral,"Thanks. I have two for Sundar, kind of the sam..."
1,Amazon,Neutral,"Amazon.com, Inc. (NASDAQ:AMZN) Q3 2018 Earning..."
2,Google,Positive,"On Pixel, look, first of all, part of the big ..."
3,Google,Positive,"But there are, you're right, the gating factor..."
4,Google,Positive,"So in terms of your first question, we actuall..."


In [12]:
with open('config.json','r') as f:
    config_data = f.read()

config = json.loads(config_data)

In [13]:
#Introducing IBM API:
from watson_developer_cloud import NaturalLanguageUnderstandingV1
from watson_developer_cloud.natural_language_understanding_v1 import Features, SentimentOptions

natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2018-03-16',
    iam_apikey=config['IBM API Key'],#need private key here
    url=config['IBM API url']
)

In [18]:
ray.shutdown()

In [28]:
@ray.remote
def sentiment(t):
	if (len(t)<15 or len(t.split(' '))<3):
		t = t + "This is neutral"
	sentimentData = natural_language_understanding.analyze(text=t,features=Features(sentiment=SentimentOptions(targets=['']))).get_result()
	row = [i, 'IBM', sentimentData['sentiment']['document']['label'],sentimentData['sentiment']['document']['score']]
	return row

TypeError: can't pickle thread.lock objects